In [1]:
import polars as pl

# Cambiar extensión de .txt a .csv
ka1 = pl.read_csv("./exp/exp49422/kaggle/KA49422_10000.csv")
ka2 = pl.read_csv("./exp/exp4966/kaggle/KA4966_11000.csv")

# Filtrar clientes que tienen Predicted == 1
cli1 = set(ka1.filter(pl.col("Predicted") == 1)["numero_de_cliente"].to_list())
cli2 = set(ka2.filter(pl.col("Predicted") == 1)["numero_de_cliente"].to_list())

# Intersección
coinciden = cli1 & cli2

print(f"Cantidad de clientes con Predicted=1 que coinciden en ambos envíos: {len(coinciden)}")


Cantidad de clientes con Predicted=1 que coinciden en ambos envíos: 7412


In [2]:
# Renombrar columnas para distinguir después del join
ka1 = ka1.rename({"Predicted": "Predicted_49422"})
ka2 = ka2.rename({"Predicted": "Predicted_4966"})

# Hacer join por numero_de_cliente
df_merged = ka1.join(ka2, on="numero_de_cliente", how="outer")

# Crear nueva columna: 1 cuando AL MENOS UNO predice 1 (OR)
df_final = df_merged.with_columns([
    pl.when(
        (pl.col("Predicted_49422") == 1) | (pl.col("Predicted_4966") == 1)
    )
    .then(1)
    .otherwise(0)
    .alias("Predicted")
])

# Seleccionar solo las columnas del formato original
df_final = df_final.select(["numero_de_cliente", "Predicted"])

# Guardar CSV
df_final.write_csv("./exp/KA_union_49422_vs_4966.csv")

# Estadísticas
total_clientes = len(df_final)
clientes_pred_1 = (df_final["Predicted"] == 1).sum()

print(f"CSV generado: KA_union_49422_vs_4966.csv")
print(f"   Total clientes: {total_clientes:,}")
print(f"   Predicted = 1: {clientes_pred_1:,}")
print(f"   Predicted = 0: {total_clientes - clientes_pred_1:,}")

/tmp/ipykernel_1880949/3917391916.py:6: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  df_merged = ka1.join(ka2, on="numero_de_cliente", how="outer")


CSV generado: KA_union_49422_vs_4966.csv
   Total clientes: 164,313
   Predicted = 1: 13,588
   Predicted = 0: 150,725
